In [ ]:
!pip install pyarrow

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving clean_data.parquet to clean_data (1).parquet


In [ ]:
import pandas as pd

df = pd.read_parquet('clean_data.parquet', engine='pyarrow')
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2019-12-18 15:52:30,2019-12-18 15:54:39,N,1.0,264,264,5.0,0.00,3.5,0.50,0.5,0.01,0.0,0.3,4.81,1.0,1.0,0.00
1,2,2020-01-01 00:45:58,2020-01-01 00:56:39,N,5.0,66,65,2.0,1.28,20.0,0.00,0.0,4.06,0.0,0.3,24.36,1.0,2.0,0.00
2,2,2020-01-01 00:41:38,2020-01-01 00:52:49,N,1.0,181,228,1.0,2.47,10.5,0.50,0.5,3.54,0.0,0.3,15.34,1.0,1.0,0.00
3,1,2020-01-01 00:52:46,2020-01-01 01:14:21,N,1.0,129,263,2.0,6.30,21.0,3.25,0.5,0.00,0.0,0.3,25.05,2.0,1.0,2.75
4,1,2020-01-01 00:19:57,2020-01-01 00:30:56,N,1.0,210,150,1.0,2.30,10.0,0.50,0.5,0.00,0.0,0.3,11.30,1.0,1.0,0.00


In [ ]:
print(f'Total rows: {df.shape[0]}')
print(f'Total columns: {df.shape[1]}')

Total rows: 3876894
Total columns: 19


In [ ]:
def find_outliers_iqr(column):
    Q1 = column.quantile(0.25)  # First quartile (25th percentile)
    Q3 = column.quantile(0.75)  # Third quartile (75th percentile)
    IQR = Q3 - Q1  # Interquartile range
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return (column < lower_bound) | (column > upper_bound)

numeric_cols = df.select_dtypes(include=['number']).columns

numeric_cols = [col for col in numeric_cols if col != 'DOLocationID']

outlier_summary = []
total_outliers = 0
outlier_mask = pd.Series(False, index=df.index)

for col in numeric_cols:
    outliers = find_outliers_iqr(df[col])
    count = outliers.sum()
    outlier_summary.append({"Column Name": col, "Outlier Count": count})
    total_outliers += count
    outlier_mask |= outliers

outlier_summary_df = pd.DataFrame(outlier_summary)

print("\nSummary of Outliers in Numerical Columns:\n")
print(outlier_summary_df.to_string(index=False))

print(f"\nTotal Number of Outliers: {total_outliers}")

no_outliers_df = df[~outlier_mask].reset_index(drop=True)

print(f"\nShape of the new dataset without outliers: {no_outliers_df.shape}")


Summary of Outliers in Numerical Columns:

          Column Name  Outlier Count
             VendorID         598286
           RatecodeID         171845
         PULocationID         321269
      passenger_count         555639
        trip_distance         347300
          fare_amount         289396
                extra         165288
              mta_tax         337842
           tip_amount         143485
         tolls_amount         131366
improvement_surcharge           4075
         total_amount         260034
         payment_type           7483
            trip_type         142015
 congestion_surcharge         921579

Total Number of Outliers: 4396902

Shape of the new dataset without outliers: (1673170, 19)


In [ ]:
no_outliers_df.to_csv('/content/no_outliers_data.csv', index=False)
files.download('/content/no_outliers_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>